In [ ]:
import os

os.chdir('/content/LDA_data')
data = os.listdir()

In [ ]:
! python -m spacy download ru_core_news_sm

In [ ]:
import nltk as nltk
import re
from string import punctuation
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from nltk.corpus import stopwords




stopwords = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    # nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out



def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        new_1 = [token for token in new if token not in stopwords]
        final.append(new_1)
    return (final)







In [ ]:
search_list = ['облигация', 'процент', 'проц', 'займ', 'заем', 'фонд', 'бумага', 'дисконт', 
               'купон', 'рента', 'ипотечныи', 'ценности', 'доходность', 'котировка', 'эмиссия',
               'баланс', 'дивиденд', 'прибыль', 'акционерныи', 'капитал', 'акция', 'акционер', 'собрание', 'правление']
search_list

In [ ]:
! pip install pyLDAvis

In [ ]:
from gensim.test.utils import datapath
import re
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


issues_with_page = []
lda_models = []
lda_topics = []

counter = 0
for text in data:
    print(text)
    print(counter)
    os.chdir('/content/LDA_data')
    f = open(text, encoding='Windows-1251', mode='r')
    issue_text = f.read()
    f.close()
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(issue_text)])
    data_all = []
    for s in sentences:
       data_all.append(s)

    lemmatized_texts = lemmatization(data_all)
    data_words = gen_words(lemmatized_texts)
    new_data_text = list(data_words)
    id2word = corpora.Dictionary(new_data_text)
    texts = new_data_text
    corpus = [id2word.doc2bow(text) for text in texts]

    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=50,
                                            passes=10,
                                            alpha="auto")
    
    list_of_topics = lda_model.print_topics()
    for topic in list_of_topics: 
        string = topic[1]
        for word in search_list:
            index = string.find(word)
            word_weight = 0
            if index != -1:
                try: 
                    word_weight = float(string[index-7:index-2])
                except:
                        continue
            if word_weight > 0.010:
                issues_with_page.append(text[:-4])
                lda_topics.append(lda_model.print_topics())
    

    # os.chdir('/content/LDA_models')
    # vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
    # pyLDAvis.save_html(vis, f"{text[:-4]}" + '_lda_model.html')

    counter += 1




    


    


  

    

    

In [ ]:
issues_with_page_dict = issues_with_page
lda_topics_dict = lda_topics
dict_to_pd = dict(zip(issues_with_page_dict, lda_topics_dict))


In [ ]:
import pandas as pd

df = pd.DataFrame(dict_to_pd)



In [ ]:
! pip install openpyxl

In [ ]:
#Сохранение отобранных LDA моделей в виде таблицы Excel

os.chdir('/content/Excel_df')
df = df.T
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer)
writer.save()

In [ ]:
!zip -r /content/LDAmodels.zip /content/Folder_To_Zip

In [ ]:
# сохранение визуализаций моделей в архив

import os
import zipfile
fantasy_zip = zipfile.ZipFile('/content/LDA_zip/archive.zip', 'w')
for folder, subfolders, files in os.walk('/content/LDA_models'):
    for file in files:
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), 'C:\\Stories\\Fantasy'), compress_type = zipfile.ZIP_DEFLATED)
fantasy_zip.close()

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=new_data_text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#Удалить папку LDA_data

import shutil 
 
dir_path = '/content/LDA_data/'

try:
    shutil.rmtree(dir_path)
except OSError as e:
    print("Error: %s : %s" % (dir_path, e.strerror))